# Tissue segmentation
Tissue segmentation is a critical step in neuroimaging to differentiate brain tissue types, such as gray matter (GM), white matter (WM), and cerebrospinal fluid (CSF), for accurate analysis in studies like functional MRI (fMRI) or structural imaging.  We used the FMRIB’s Automated Segmentation Tool (FAST), part of the FSL (FMRIB Software Library), to perform this segmentation. 
The resulting tissue maps will be used in the next step for artifact removal.

-----------------------------------------------------------
Script written by Sepideh Tabrik & Mehdi Behroozi
<br>
Biopsychology, 
<br>
Ruhr-University Bochum, Bochum, Germany
<br>
(2022.03)

-----------------------------------------------------------

In [ ]:
import glob
import os
from pathlib import Path

# Define data path
data_path = Path("/mnt/d/Data/Human/ObjectCategorization/analysis2")

# Find subject directories
sub_dirs = list(data_path.glob("sub*/ses-*"))

# Process each subject directory
for curr_sub in sub_dirs:
    print(f"Processing: {curr_sub.parts[-2]}/{curr_sub.parts[-1]}")

    reg_dir = curr_sub / "reg"
    output = curr_sub / "tissue_segmentation"

    # Ensure the output directory exists
    output.mkdir(parents=True, exist_ok=True)

    # Run segmentation only if reg_dir exists
    if reg_dir.exists():
        os.system(f"fast -t 1 -n 3 -H 0.1 -I 4 -l 20.0 -o {output}/highres {reg_dir}/highres")
    else:
        print(f"Warning: Registration directory {reg_dir} not found, skipping.")


## apply threshould to CSF and WM masks and binarize them


In [ ]:

import glob
import os
from pathlib import Path

# Define data path
data_path = Path("/mnt/d/Data/Human/ObjectCategorization/analysis2")

# Find tissue segmentation directories
sub_dirs = list(data_path.glob("sub*/ses*/tissue_segmentation"))

# Process each directory
for curr_sub in sub_dirs:
    print(f"Processing: {curr_sub}")

    # Change directory only if it exists
    if curr_sub.exists():
        os.chdir(curr_sub)
    else:
        print(f"\tWarning: {curr_sub} does not exist, skipping.")
        continue

    # Define input files
    csf_file = curr_sub / "highres_pve_0.nii.gz"
    wm_file = curr_sub / "highres_pve_2.nii.gz"

    # Apply threshold and binarize only if the files exist
    if csf_file.exists():
        print("\tApplying threshold and binarizing CSF mask...")
        os.system(f"fslmaths {csf_file} -thr 0.99 -div {csf_file} {curr_sub}/CSF.nii.gz")
    else:
        print(f"\tWarning: {csf_file} not found, skipping CSF processing.")

    if wm_file.exists():
        print("\tApplying threshold and binarizing WM mask...")
        os.system(f"fslmaths {wm_file} -thr 0.99 -div {wm_file} {curr_sub}/WM.nii.gz")
    else:
        print(f"\tWarning: {wm_file} not found, skipping WM processing.")


## convert csf and wm mask from anatomical space to functional space


In [ ]:
from pathlib import Path
import os

data_path = Path('/mnt/d/Data/Human/ObjectCategorization/analysis2')
sub_dirs = list(data_path.glob('sub*/ses*/func/rest*/'))



for curr_sub in sub_dirs:
    print(curr_sub)

    # Define paths using pathlib
    reg_trans = curr_sub / 'reg/highres2example_func.mat'
    example_func = curr_sub / 'reg/example_func_brain.nii.gz'
    tissue_seg = curr_sub.parents[1] / 'tissue_segmentation'  # More robust path extraction
    
    # Apply transformations using FLIRT
    os.system(f'flirt -in {tissue_seg}/CSF -ref {example_func} -applyxfm -interp nearestneighbour -init {reg_trans} -out {curr_sub}/CSF')
    os.system(f'flirt -in {tissue_seg}/WM -ref {example_func} -applyxfm -interp nearestneighbour -init {reg_trans} -out {curr_sub}/WM')
